In [30]:
from pynq import Bitstream, MMIO
from time import sleep

bit = "/home/xilinx/jupyter_notebooks/xilinx/overlays/nec/design_1_wrapper.bit"
Bitstream(bit).download()

# AXI GPIO base addresses (verify these match your design's address map)
GPIO0_BASE = 0x41200000  # addr/cmd GPIO
GPIO1_BASE = 0x41210000  # strobe GPIO

gpio0 = MMIO(GPIO0_BASE, 0x10000)
gpio1 = MMIO(GPIO1_BASE, 0x10000)

# AXI GPIO register offsets (per IP)
CH1_DATA = 0x00
CH1_TRI  = 0x04
CH2_DATA = 0x08
CH2_TRI  = 0x0C

# Set directions: all outputs
gpio0.write(CH1_TRI, 0x00000000)  # channel 1 (addr) out
gpio0.write(CH2_TRI, 0x00000000)  # channel 2 (cmd) out
gpio1.write(CH1_TRI, 0x00000000)  # channel 1 (strobe) out

def pulse(mmio, data_ofs, width_us=50):
    # If your logic is edge-triggered, the exact width is not critical.
    mmio.write(data_ofs, 1)
    # Ensure the write posts; a tiny delay helps and is more reliable than tens of microseconds in Python.
    sleep(max(width_us/1e6, 0.001))  # at least 1 ms for reliability
    mmio.write(data_ofs, 0)

def nec_send_cmd(addr, cmd, send_pulse_us=50, inter_frame_ms=100):
    gpio0.write(CH1_DATA, addr & 0xFF)   # address byte
    gpio0.write(CH2_DATA, cmd  & 0xFF)   # command byte
    pulse(gpio1, CH1_DATA, send_pulse_us)  # strobe on second GPIO
    sleep(inter_frame_ms/1e3)
    
def cmd_table(cmd_to_nec):
    match cmd_to_nec:
        case '0':  return nec_send_cmd(0x68,0x00)
        case '1':  return nec_send_cmd(0x30,0x00)
        case '2':  return nec_send_cmd(0x18,0x00)
        case '3':  return nec_send_cmd(0x7A,0x00)
        case '4':  return nec_send_cmd(0x10,0x00)
        case '5':  return nec_send_cmd(0x38,0x00)
        case '6':  return nec_send_cmd(0x5A,0x00)
        case '7':  return nec_send_cmd(0x42,0x00)
        case '8':  return nec_send_cmd(0x4A,0x00)
        case '9':  return nec_send_cmd(0x52,0x00)
        case '-':  return nec_send_cmd(0xE0,0x00)
        case '+':  return nec_send_cmd(0xA8,0x00)

cmd_table('-')
sleep(4)
cmd_table('0')
sleep(4)
cmd_table('0')
sleep(4)
cmd_table('0')
sleep(4)
cmd_table('5')
sleep(4)
cmd_table('+')
sleep(4)
print("Finish")

Finish
